# simInvNB Using the GPU

## Crude Helper Function

In [2]:
using Distributions
using LinearAlgebra
using RCall

┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1273
┌ Info: Precompiling RCall [6f49c342-dc21-5d91-9882-a32aef131414]
└ @ Base loading.jl:1273


In [3]:
function cov2cor(M)
    Mcor = copy(M)
    @simd for i in CartesianIndices(M)
        Mcor[i] = M[i] / sqrt(M[i[1], i[1]] * M[i[2], i[2]])
    end
    return Mcor
end

cov2cor (generic function with 1 method)

In [4]:
function rcor(d, ρ_constant=false)
    if ρ_constant
        ρ_temp = rand(Uniform(0.25, 0.90))
        Σ = fill(ρ_temp, (d, d))
        @simd for i = 1:d
            @inbounds Σ[i,i] = 1.0
        end
    else
        tmp  = randn(d, d)
        mcov = tmp * tmp'
        Σ    = cov2cor(mcov)
    end
    return Σ
end     

rcor (generic function with 2 methods)

In [5]:
function rmvnb(d::Int)
    rates = rand(1:5, d)
    probs = rand(0.2:0.01:0.8, d)
    return collect((NegativeBinomial, (rate, prob)) for (rate, prob) in zip(rates, probs))
end

rmvnb (generic function with 1 method)

In [6]:
rmvnb(4)

4-element Array{Tuple{UnionAll,Tuple{Int64,Float64}},1}:
 (NegativeBinomial, (3, 0.41))
 (NegativeBinomial, (3, 0.21))
 (NegativeBinomial, (5, 0.23))
 (NegativeBinomial, (3, 0.55))

## Helper Functions

In [7]:
function convertCor(ρ, from, to)
    F = Dict(
        ("pearson", "pearson")  => A -> A,
        ("pearson", "spearman") => A -> (6 / pi) .* asin.(A ./ 2),
        ("pearson", "kendall")  => A -> (2 / pi) .* asin.(A),
        ("spearman", "pearson") => A -> 2 .* sin.(A .* (pi / 6)),
        ("spearman", "spearman")=> A -> A,
        ("spearman", "kendall") => A -> (2 / pi) .* asin.(2 .* sin.(A .* (pi / 6))),
        ("kendall", "pearson")  => A -> sin.(A .* (pi / 2)),
        ("kendal", "spearman")  => A -> (6 / pi) .* asin.(sin.(A .* (pi / 2)) ./ 2),
        ("kendall", "kendall")  => A -> A
    )
    tmp = F[(from, to)](ρ)
    @rput tmp
    return rcopy(R"as.matrix(Matrix::nearPD(tmp)$mat)")
end

convertCor (generic function with 1 method)

In [14]:
function adjustCor(ρ, params)
    d = size(ρ, 1)
    
    indexMat = rcopy(R"combn(1:$d, 2)")
    ρ_out = Matrix{Float64}(I, d, d)
    
    iter = [(j,i) for (j,i) in enumerate(eachcol(indexMat))]
    Threads.@threads for it in iter
        j = it[1]
        i = it[2]
        α₁ = params[:α][i[1]]
        α₂ = params[:α][i[2]]
        λ₁ = params[:λ][i[1]]
        λ₂ = params[:λ][i[2]]
        σ₁ = sqrt(params[:σ][i[1]])
        σ₂ = sqrt(params[:σ][i[2]])
        
        scaleFactor = sqrt(α₁ * α₂)
        ρ_tmp = ρ[i...]
        ρ_targetGamma = ρ_tmp * (σ₁ * σ₂) / (scaleFactor * λ₁ * λ₂)
        @inbounds ρ_out[i...] = ρ_out[reverse(i)...] = min(1, ρ_targetGamma)
    end
    
    eigenDecomp = eigen(ρ_out)
    if any(eigenDecomp.values .< 0)
        @rput ρ_out
        return cov2cor(rcopy(R"as.matrix(Matrix::nearPD(ρ_out)$mat)"))
    end
    
    return ρ_out
end

adjustCor (generic function with 1 method)

In [15]:
# margin = (Distribution, (param1, param2, ...))
function normal2marginal(x, margin)
    quantile.(margin[1](margin[2]...), cdf.(Normal(), x))
end

normal2marginal (generic function with 1 method)

## Simulate Negative Binomial

In [16]:
function rvec(n, ρ, margins, inputCorType)
    ρ = convertCor(ρ, inputCorType, "pearson")
    d = size(ρ, 1)

    mvn = rand(MultivariateNormal(zeros(d), ρ), n)'
    
    Threads.@threads for i in 1:d
        mvn[:,i] = normal2marginal(mvn[:,i], margins[i])
    end
    
    return mvn
end

rvec (generic function with 1 method)

# Testing

In [52]:
using Test

In [85]:
d = 100
n = Int(1e7)
ρ = rcor(d);
margins = rmvnb(d)
s1 = rvec(n, ρ, margins, "pearson")
ρ_hat = cor(s1)

100×100 Array{Float64,2}:
  1.0          0.0859355    -0.0648805   …   0.0257976    -0.064315 
  0.0859355    1.0           0.00921852     -0.0355429    -0.218549 
 -0.0648805    0.00921852    1.0             0.0561855     0.122769 
  0.078731     0.007477     -0.0218245       0.00914989    0.0516017
 -0.202522     0.0191046    -0.127106       -0.00806571    0.0314712
 -0.0101051   -0.0621611    -0.11505     …   0.0519673    -0.0572856
  0.0860626    0.0547228     0.0838671       0.000971664  -0.0673425
  0.104438     0.0794737    -0.0288445      -0.00916851    0.094365 
  0.00551062   0.0954886     0.0813635       0.0927361    -0.015372 
 -0.0963952   -0.0559465     0.0326044      -0.0367154     0.0280819
 -0.017649     0.0593911    -0.106748    …  -0.0783684    -0.0631905
 -0.0587731    0.0774242    -0.0874482       0.0544168    -0.0867599
 -0.0456822    0.0073854    -0.0310187      -0.065102     -0.023483 
  ⋮                                      ⋱                          
  0.0584

In [90]:
ρ_hat[findall(abs.(ρ .- ρ_hat) .> 0.1)]

4-element Array{Float64,1}:
 -0.20081011101530913
 -0.1799820979399502 
 -0.1799820979399502 
 -0.20081011101530913

In [91]:
ρ[findall(abs.(ρ .- ρ_hat) .> 0.1)]

4-element Array{Float64,1}:
 -0.31579970592528367
 -0.30785828435375434
 -0.30785828435375434
 -0.31579970592528367